In [1]:
import os

In [2]:
%pwd

'd:\\Data Science\\Customer Churn Prediction with MLOps\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Data Science\\Customer Churn Prediction with MLOps'

In [5]:
import dagshub
dagshub.init(repo_owner='vaibhavpandya2903', repo_name='customer-churn-prediction', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as vaibhavpandya2903

Initialized MLflow to track repo "vaibhavpandya2903/customer-churn-prediction"

Repository vaibhavpandya2903/customer-churn-prediction initialized!

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XGBoost
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/vaibhavpandya2903/customer-churn-prediction.mlflow",
           
        )

        return model_evaluation_config


In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [12]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        # Handle NaN values
        test_x = test_x.fillna(test_x.mean())  # Use mean imputation
        test_y = test_y.fillna(test_y.mean())

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            # Save metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Register the model if using a remote MLflow tracking server
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="XGBClassifierModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [13]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-02-06 15:36:39,997: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-06 15:36:39,999: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-06 15:36:40,001: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-06 15:36:40,003: INFO: common: created directory at: artifacts]
[2025-02-06 15:36:40,003: INFO: common: created directory at: artifacts/model_evaluation]
[2025-02-06 15:36:40,437: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'XGBClassifierModel' already exists. Creating a new version of this model...
2025/02/06 15:36:49 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: XGBClassifierModel, version 4
Created version '4' of model 'XGBClassifierModel'.
